In [3]:
%pip install -q --no-cache-dir --force-reinstall airavata-python-sdk[notebook]
import airavata_jupyter_magic

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 23.2.0 requires cryptography!=40.0.0,!=40.0.1,<42,>=38.0.0, but you have cryptography 44.0.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.

Loaded airavata_jupyter_magic (2.0.12) 
(current runtime = local)

  %authenticate                      -- Authenticate to access high-performance runtimes.
  %request_runtime <rt> [args]       -- Request a runtime named <rt> with configuration <args>. Call multiple times to request multiple runtimes.
  %restart_runtime <rt>              -- Restart runtime <rt>. Run this if you install new dependencies or if the runtime hangs.
  %stop_runtime <rt>                 -- Stop runtime <rt> when no longer needed.
  %switch_runtime <rt>               -- Switch active runtime to <rt>. All subsequent executions will use this runtime.


In [4]:
%pip install 'transformers[torch]' streamlit matplotlib

Note: you may need to restart the kernel to use updated packages.


In [10]:
%authenticate
%request_runtime hpc_cpu --file=cybershuttle.yml --walltime=60 --use=NeuroData25VC1:cloud,expanse:shared,anvil:shared
%switch_runtime hpc_cpu

Output()

Authenticated.

Requesting runtime=hpc_cpu
cpuCount: 4
experimentName: CS_Agent
group: Default
libraries:
- python=3.10
- pip
memory: 4096
mounts: []
nodeCount: 1
pip:
- streamlit
- transformers
- torch
- matplotlib
queue: cloud
remoteCluster: NeuroData25VC1
wallTime: 60

Requested runtime=hpc_cpu. state=CONFIGURING_WORKSPACE
Switched to runtime=hpc_cpu.


In [11]:
import sys
import time
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification


start = time.time()
# Get input text from command-line argument
user_input = "The protagonist faced internal conflict." #sys.argv[1]

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english", trust_remote_code=True)
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

# Run classification
result = classifier(user_input)[0]

# Print result to stdout
print(f"Label: {result['label']}")
print(f"Confidence: {result['score']:.4f}")
end = time.time()
print(f"Inference time: {end - start:.2f} seconds")

⠇ Connecting to=hpc_cpu... status=CONNECTEDWORKSPACE


Label: NEGATIVE
Confidence: 0.9471
Inference time: 3.01 seconds


In [12]:
import streamlit as st
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import matplotlib.pyplot as plt

# Page title
st.set_page_config(page_title="SAT-2 Text Classifier", layout="centered")
st.title("📚 SAT-2 English Text Classifier")
st.write(
    "This app uses a fine-tuned DistilBERT model to classify text related to SAT-2 English exam topics."
)

# Text input
user_input = st.text_area("Enter your text below:", height=200)

# Help and examples
with st.expander("💡 Help & Examples"):
    st.markdown(
        """
    - This model is fine-tuned to recognize topics in English SAT-2 exam texts.
    - Try pasting a paragraph from a literature article or grammar question.
    - **Example Input:** _'The protagonist's moral struggle illustrates the conflict between personal duty and societal norms.'_
    """
    )

# Button to run classification
if st.button("Classify"):
    if user_input.strip() == "":
        st.error("❗ Please enter some text to classify.")
    else:
        with st.spinner("Classifying..."):
            try:
                # Load model and tokenizer
                tokenizer = AutoTokenizer.from_pretrained(
                    "distilbert-base-uncased-finetuned-sst-2-english"
                )
                model = AutoModelForSequenceClassification.from_pretrained(
                    "distilbert-base-uncased-finetuned-sst-2-english",
                    trust_remote_code=True,
                )

                # Prepare pipeline
                classifier = pipeline(
                    "text-classification", model=model, tokenizer=tokenizer
                )

                # Perform classification
                result = classifier(user_input)[0]

                # Display results
                st.success("✅ Classification Result")
                st.markdown(f"### 🏷️ Predicted Label: `{result['label']}`")
                st.markdown(f"### 🔢 Confidence Score: `{result['score']:.4f}`")

                # Optional bar visualization
                st.bar_chart({"Confidence": [result["score"]]})

            except Exception as e:
                st.error("⚠️ An error occurred during classification.")
                st.text(f"Details: {e}")


⠸ Connecting to=hpc_cpu... status=CONNECTED


In [15]:
%streamlit run app.py

⠼ Connecting to=hpc_cpu... status=CONNECTED


In [ ]:
!streamlist run app.py

In [23]:
!export TOKENIZERS_PARALLELISM=false

⠸ Connecting to=hpc_cpu... status=CONNECTED
